# 단어 수준의 번역기 만들기

## 참고

https://wikidocs.net/86900


## 데이터 출처

http://www.manythings.org/anki/  
fra-eng.zip의 fra.txt 파일을 사용

In [1]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
num_samples = 20000 # 2만개의 샘플만 사용

## Preprocessing

In [3]:
def to_ascii(s):
    # 프랑스어 악센트(accent) 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백 추가.
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [4]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


전체 2만개의 샘플에 대해 전처리를 수행하고 훈련 과정에서는 교사 강요를 사용.  
입력 시퀀스에는 시작을 의미하는 \<sos\>토큰, 출력 시퀀스에는 종료를 의미하는 \<eos\>토큰을 추가.

In [16]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    with open("fra.txt", "r", encoding='utf-8') as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [17]:
# 3개의 데이터 셋 인코더의 입력, 디코더의 입력, 디코더의 레이블
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>']]


글자 수준의 seq2seq과 마찬가지로 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고 이전 시점의 실제 값을 넣어준다. 따라서 sents_fra_in이 필요하다!! (교사 강요)

### 정수 인코딩

In [18]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [19]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (20000, 7)
디코더의 입력의 크기(shape) : (20000, 15)
디코더의 레이블의 크기(shape) : (20000, 15)


In [20]:
# 단어 집합의 크기
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 3368, 프랑스어 단어 집합의 크기 : 6030


In [21]:
# 단어로부터 정수를 얻는 딕셔너리, 정수로부터 단어를 얻는 딕셔너리
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [22]:
# 테스트 데이터로 분리 전 데이터 섞기 ( 순서가 섞인 정수 시퀀스 리스트)
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [ 3097  9811 18795 ...  4343 10017 13258]


In [23]:
# 샘플 섞기
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [24]:
# 임의의 샘플 출력
print(encoder_input[10721])
print(decoder_input[10721])
print(decoder_target[10721])

[  6   9 373   1   0   0   0]
[  2   8 220   1   0   0   0   0   0   0   0   0   0   0   0]
[  8 220   1   3   0   0   0   0   0   0   0   0   0   0   0]


8, 220, 1 이라는 동일 시퀀스 확인 ( 2와 3은 \<sos\>, \<eos\> )

In [26]:
n_of_val = int(20000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 2000


In [27]:
# train_test_split
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [28]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (18000, 7)
훈련 target 데이터의 크기 : (18000, 15)
훈련 target 레이블의 크기 : (18000, 15)
테스트 source 데이터의 크기 : (2000, 7)
테스트 target 데이터의 크기 : (2000, 15)
테스트 target 레이블의 크기 : (2000, 15)


## 기계 번역기 만들기

In [29]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [30]:
embedding_dim = 64
hidden_units = 64

인코더 설계
- Masking은 연산을 제외하는 역할
- 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state = True
- state_h : hidden state, state_c : cell state

In [31]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

다중 클래스 분류 문제이므로 출력층으로 소프트맥스 함수와 손실 함수를 크로스 엔트로피 함수를 사용한다. 크로스 엔트로피를 사용하기 위해서 레이블은 원-핫 인코딩이 되어야 한다. 이 때 원-핫 인코딩이 되지 않은 정수 레이블에 대한 다중 클래스 분류 문제를 풀고 싶을 때는 categorical_crossentropy 대신 sparse_categorical_crossentropy를 사용하면 된다.

In [32]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [33]:
# training
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=5)

Epoch 1/5
141/141 [==============================] - 18s 69ms/step - loss: 4.0931 - acc: 0.5985 - val_loss: 2.1721 - val_acc: 0.6133
Epoch 2/5
141/141 [==============================] - 7s 52ms/step - loss: 2.0266 - acc: 0.6176 - val_loss: 1.9022 - val_acc: 0.6381
Epoch 3/5
141/141 [==============================] - 7s 51ms/step - loss: 1.8016 - acc: 0.6913 - val_loss: 1.7415 - val_acc: 0.7372
Epoch 4/5
141/141 [==============================] - 7s 52ms/step - loss: 1.6653 - acc: 0.7423 - val_loss: 1.6256 - val_acc: 0.7460
Epoch 5/5
141/141 [==============================] - 7s 52ms/step - loss: 1.5536 - acc: 0.7531 - val_loss: 1.5202 - val_acc: 0.7620


## seq2seq Running

1. 번역하고자 하는 입력 문장이 인코더에 들어가서 hidden state와 cell state를 얻음
2. 2가지의 state와 \<sos\>를 디코더로 보냄
3. 디코더가 \<eos\>가 나올때까지 반복하여 다음 문자를 예측함

In [35]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

decode_sequence
1. 입력 문장이 들어오면 인코더는 마지막 시점까지 전개하여 마지막 시점의 hidden state와 cell state를 리턴
2. 두 state를 states_value에 저장하고 디코더의 초기 입력으로 \<sos\>를 준비하여 target_seq에 저장
3. 두 가지 입력을 가지고 while문으로 진입하여 두 가지를 디코더의 입력으로 사용
4. 현재 시점에 대한 예측 시작 - 현재 시점의 예측 벡터는 output_tokens, 현재 시점의 hidden state는 h, 현재 시점의 cell state는 c, 현재 시점의 예측 단어는 target_seq으로 while문의 루프를 돌며 현재 시점의 예측 단어로 \<eos\>를 예측하거나 번역 문장의 길이가 50이 넘을 때까지 반복한다. 각 시점마다 번역한 단어는 decoded_sentence에 누적하여 저장하였다가 최종 번역 시퀀스로 리턴한다.

In [36]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
            len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

결과확인을 위한 함수 seq_to_src, seq_to_tar  
영어 문장에 해당하는 정수 시퀀스를 입력받으면 영어 문장으로 변환, seq_to_tar은 프랑스어

In [37]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

훈련 데이터에서 임의로 선택한 인덱스의 샘플 결과

In [38]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

입력문장 : i felt fear . 
정답문장 : j ai ressenti de la peur . 
번역문장 : je ai ai . 
--------------------------------------------------
입력문장 : can you show me ? 
정답문장 : peux tu me montrer ? 
번역문장 : tom est . 
--------------------------------------------------
입력문장 : i got goosebumps . 
정답문장 : j ai la chair de poule . 
번역문장 : je ai ai . 
--------------------------------------------------
입력문장 : class dismissed . 
정답문장 : le cours est termine . 
번역문장 : c est . 
--------------------------------------------------
입력문장 : you re early . 
정답문장 : vous etes en avance . 
번역문장 : tom est . 
--------------------------------------------------


테스트 데이터에서 임의로 선택한 인덱스의 샘플 결과

In [39]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

입력문장 : you re forgiven . 
정답문장 : tu es pardonne . 
번역문장 : tom est . 
--------------------------------------------------
입력문장 : we did it . 
정답문장 : nous l avons fait . 
번역문장 : tom est a . 
--------------------------------------------------
입력문장 : i like this room . 
정답문장 : j aime cette chambre . 
번역문장 : je ai ai . 
--------------------------------------------------
입력문장 : i have standards . 
정답문장 : j ai des etendards . 
번역문장 : je ai ai . 
--------------------------------------------------
입력문장 : that was fast . 
정답문장 : ca a ete rapide . 
번역문장 : tom est . 
--------------------------------------------------
